In [3]:
from models.cudaconvnet import CudaConvNet

In [4]:
model = CudaConvNet()

In [9]:
model.parameters

<bound method CudaConvNet.parameters of CudaConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (lrn1): LRN(
    (average): AvgPool3d(kernel_size=(3, 1, 1), stride=1, padding=(1, 0, 0), ceil_mode=False, count_include_pad=True)
  )
  (lrn2): LRN(
    (average): AvgPool3d(kernel_size=(3, 1, 1), stride=1, padding=(1, 0, 0), ceil_mode=False, count_include_pad=True)
  )
  (fc1): Linear(in_features=1024, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
)>

In [15]:
params = [p for p in model.parameters()]
for p in params:
    print(p.size())

torch.Size([32, 3, 5, 5])
torch.Size([32])
torch.Size([32, 32, 5, 5])
torch.Size([32])
torch.Size([64, 32, 5, 5])
torch.Size([64])
torch.Size([10, 1024])
torch.Size([10])
torch.Size([10, 10])
torch.Size([10])


In [16]:
params[0]

Parameter containing:
(0 ,0 ,.,.) = 
  0.0481  0.0050  0.0952  0.0707  0.0353
 -0.0085 -0.0040  0.0332  0.0478  0.0094
 -0.0560  0.0507  0.0740 -0.0790  0.0324
 -0.0700  0.0372  0.0446  0.0074 -0.0912
 -0.1042  0.0713 -0.0285  0.0725 -0.0625

(0 ,1 ,.,.) = 
 -0.0679  0.0690 -0.0141  0.0563 -0.0042
  0.1040 -0.0861  0.1017 -0.0345  0.1020
  0.1083 -0.0275  0.0886  0.0836 -0.0858
  0.0261  0.0265  0.0835 -0.1142  0.1036
 -0.0244  0.0695  0.0014  0.0073  0.0294

(0 ,2 ,.,.) = 
  0.0881 -0.0592 -0.0594  0.0263 -0.0898
  0.0616 -0.0266 -0.0648 -0.0901 -0.0026
 -0.0254 -0.0071  0.0387 -0.0999  0.0475
 -0.0660  0.0794 -0.0802 -0.0902 -0.0145
  0.0322 -0.0129 -0.0629 -0.0973 -0.1089
     ⋮ 

(1 ,0 ,.,.) = 
  0.0686 -0.0266 -0.0188 -0.0004 -0.0814
  0.0085 -0.0626 -0.0935 -0.0716  0.0640
  0.1020 -0.0269 -0.0289  0.1126 -0.0361
 -0.0866 -0.0174  0.0772  0.0148 -0.0505
 -0.0965 -0.0582  0.0425 -0.0466 -0.0523

(1 ,1 ,.,.) = 
  0.0658  0.0072 -0.0011 -0.0528  0.0241
 -0.0450  0.1065  0.0653 -0.03

In [17]:
import torch.nn as nn
import torch.nn.functional as F

In [18]:
class LRN(nn.Module):
    def __init__(self, local_size=1, alpha=1.0, beta=0.75, ACROSS_CHANNELS=True):
        super(LRN, self).__init__()
        self.ACROSS_CHANNELS = ACROSS_CHANNELS
        if ACROSS_CHANNELS:
            self.average=nn.AvgPool3d(kernel_size=(local_size, 1, 1),
                    stride=1,
                    padding=(int((local_size-1.0)/2), 0, 0))
        else:
            self.average=nn.AvgPool2d(kernel_size=local_size,
                    stride=1,
                    padding=int((local_size-1.0)/2))
        self.alpha = alpha
        self.beta = beta


    def forward(self, x):
        if self.ACROSS_CHANNELS:
            div = x.pow(2).unsqueeze(1)
            div = self.average(div).squeeze(1)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        else:
            div = x.pow(2)
            div = self.average(div)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        x = x.div(div)
        return x


In [19]:
class CudaConvNet(nn.Module):
    def __init__(self, alpha=0.00005, beta=0.010001):
        super(CudaConvNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 5, 1, 2)  # n_input, n_output, ks, stride, padding
        self.conv2 = nn.Conv2d(32, 32, 5, 1, 2)
        self.conv3 = nn.Conv2d(32, 64, 5, 1, 2)

        self.lrn1 = LRN(3, alpha, beta)  # local size = 3
        self.lrn2 = LRN(3, alpha, beta)

        self.fc1 = nn.Linear(64 * 4 * 4, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(F.max_pool2d(out, 3, 2, 1), inplace=True)  # ks, stride, padding
        out = self.lrn1(out)

        out = F.relu(self.conv2(out), inplace=True)
        out = F.avg_pool2d(out, 3, 2, 1)
        out = self.lrn2(out)

        out = F.relu(self.conv3(out), inplace=True)
        out = F.avg_pool2d(out, 3, 2, 1)

        out = out.view(out.size(0), -1)

        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [29]:
foo = [p for p in model.conv1.parameters()]

In [35]:
foo[0].size()

torch.Size([32, 3, 5, 5])

In [38]:
import torch
from torch.autograd import Variable
a = Variable(torch.FloatTensor(1), requires_grad=True)

In [39]:
a

Variable containing:
 0
[torch.FloatTensor of size 1]

In [40]:
a = a + 5

In [41]:
a

Variable containing:
 5
[torch.FloatTensor of size 1]

In [42]:
a = Variable(torch.FloatTensor(1), requires_grad=True)
a

Variable containing:
 0
[torch.FloatTensor of size 1]

In [ ]:
a = a + 5